In [ ]:
!pip install -q transformers diffusers accelerate torch==1.13.1

In [ ]:
!pip install -q "ipywidgets>=7,<8" ftfy

# Authenticate with hugging face.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Import Dependencies

In [ ]:
rom diffusers import StableDiffusionImg2ImgPipeline, EulerDiscreteScheduler
from pathlib import Path
from PIL import Image
import torch
import re

# Remove non-word characters and foreign characters.

In [ ]:
def slugify(text):
  text = re.sub(r"[^\w\s]", "", text)
  text = re.sub(r"\s+", "-", text)
  return text

# Define Model

In [ ]:
model_id = "stabilityai/stable-diffusion-2"

# Create a list of images and put them in proper format/size

In [ ]:
images = [
    "803c6bf16e1d86b997796ebb8b4b2152.jpg",
    "a21dcfb4aa17f2b4c0d22ba91549b7db.jpg",
    "92c537eed971d0111cd63ddf4589d77b.jpg"
    ]

In [ ]:
init_images = [Image.open(image).convert("RGB").resize((768,768)) for image in images]

# Define prompts with negative prompts.

Since we will generate 3 images for this example, we will create a list of 3 images and then a corresponding list of 3 prompts.

In [ ]:
prompts = [
    "construction worker in snowy landscape",
    "construction worker in dark evening",
    "construction worker in rain storm"
    ]

negative_prompts = [
    "blurry, dark photo, blue",
    "blurry, dark photo, blue",
    "blurry, dark photo, blue"
    ]

# Create scheduler and pipeline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Use the Euler scheduler here instead of default
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)

pipe = pipe.to(device)

# Create dir for storing generated images

In [ ]:
DIR_NAME="./images/"
dirpath = Path(DIR_NAME)

# create parent dir if doesn't exist
dirpath.mkdir(parents=True, exist_ok=True)

# Define pipeline parameters and generate images based on prompts

In [ ]:
steps = 20

scale = 9

num_images_per_prompt = 1

seed = torch.randint(0, 1000000, (1,)).item()

generator = torch.Generator(device=device).manual_seed(seed)

output = pipe(
    prompts, negative_prompt=negative_prompts, image=init_images, num_inference_steps=steps,
    guidance_scale=scale, num_images_per_prompt=num_images_per_prompt, generator=generator
    )

# Iterate through and push generated images to images dir

In [ ]:
for idx, (image,prompt) in enumerate(zip(output.images, prompts*num_images_per_prompt)):
    image_name = f'{slugify(prompt)}-{idx}.png'
    image_path = dirpath / image_name
    image.save(image_path)